# Practical 1

### [Christian Cabrera](https://www.cst.cam.ac.uk/people/chc79), University

of Cambridge

### [Eric Meissner](https://www.cst.cam.ac.uk/people/erm70), University

of Cambridge

### [Neil D. Lawrence](http://inverseprobability.com), University of

Cambridge

### 2022-11-01

**Abstract**: In this lab session we look at setting up a SQL server and
making joins between different data sets.

$$
$$

<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!---->
<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!-- The last names to be defined. Should be defined entirely in terms of macros from above-->
<!--

-->

Check Session for this Practical is 3rd November 2022